# これまでの解析のデータの統合

## 準備

In [ ]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count

# モジュールのインポート
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 変数の設定

In [ ]:
intron_file = '../data/intron/human_ttn_intron.tsv'
gbk_file = '../data/gbk/human_ttn.gb'

# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk(gbk_file)

# 当該遺伝子の染色体上の位置を確認
with open(gbk_file,'r') as f:
    for line in f:
        if "ACCESSION" in line:
            acc = line.split()[3]
            break
start, end = map(int, (acc.replace("complement(","").replace(")","")).split(".."))

## intron_featureのデータ整理

In [ ]:
# データの読み込み
bed = pd.read_csv(intron_file, sep='\t', header=None, usecols=[1,2,5])

# メジャーなストランドの確認
major_strand = bed[5].value_counts().idxmax()

# メジャーなストランドのみを抽出
bed = bed[bed[5] == major_strand]
bed = bed.loc[:,[1,2]]

# 遺伝子領域外のイントロンを除外
bed = bed[(start < bed[1]) & (bed[2] < end)]

#
if major_strand == '-':
    bed = bed.apply(lambda x: end-x)
    bed = bed.loc[:,[2,1]]
    bed.columns = ["start","end"]
    bed = bed.sort_values('start')
else:
    bed = bed.apply(lambda x: x-start+1)
    bed.columns = ["start","end"]
    bed = bed.sort_values('start')

# イントロンの位置の辞書を作成
intron_dic = {}
for n,l in enumerate(gbk.intron_list()):
    intron_dic[n+1] = l

# イントロの位置情報をもとに、イントロン番号を取得
def start_pos(x):
    for k,v in intron_dic.items():
        if x == v[0]:
            return k
def end_pos(x):
    for k,v in intron_dic.items():
        if x == v[1]:
            return k
        
bed['start_pos'] = bed['start'].apply(start_pos)
bed['end_pos'] = bed['end'].apply(end_pos)
bed = bed.dropna()
bed['start_pos'].astype(int)
bed['end_pos'].astype(int)
bed['intron_skip'] = bed['end_pos'] - bed['start_pos']

## イントロの起点ののデータ整理

In [ ]:
# 5'側を起点にイントロンの組み合わせの数をカウント
s = bed.value_counts('start_pos').to_dict()
for i in range(gbk.intron_num()):
    if i+1 not in s.keys():
        s[i+1] = 0
sdf = pd.DataFrame.from_dict(s, orient='index').sort_index()


## イントロンの終点のデータの整理

In [ ]:

# 3'側を終点にイントロンの組み合わせの数をカウント
e = bed.value_counts('end_pos').to_dict()
for i in range(gbk.intron_num()):
    if i+1 not in e.keys():
        e[i+1] = 0
edf = pd.DataFrame.from_dict(e, orient='index').sort_index()


## 各エクソンの塩基数が3の倍数かどうかのデータの整理

In [ ]:

# エクソン配列が3の倍数であるエクソンの表示
exon_3x = []
for i in range(gbk.exon_num()):
    if len(gbk.exon_seq(i+1)) % 3 == 0:
        exon_3x.append(1)
    else:
        exon_3x.append(0.1)
df_3x = pd.DataFrame({"exon":[i+1 for i in range(gbk.exon_num())], 
                   "existence":exon_3x, 
                   "length":[len(gbk.exon_seq(i+1)) for i in range(gbk.exon_num())], 
                   "bar": [1 for i in range(gbk.exon_num())]})


## イントロンのスプライスバリアントのデータの整理


In [ ]:

# イントロンのスプライスバリアントの作成
skip_bed = bed[bed['intron_skip'] > 1]
skip_bed = skip_bed.sort_values('intron_skip', ascending=False)
skip_bed = skip_bed.sort_values('start_pos')

numlst = [[i,j] for i,j in zip(skip_bed['start_pos'], skip_bed['end_pos'])]
result = {0:[numlst[0]]}

for i in numlst[1:]:
    for j in range(len(result.keys())):
        if result[j][-1][1] < i[0]:
            result[j].append(i)
            break
    else:
        result.setdefault(max(result.keys())+1,[i])           
x_arr = np.array([])
y_arr = np.array([])
for i in range(len(result.keys())):
    x_arr = np.concatenate([x_arr, np.array( [i + [None] for i in result[i]]).flatten()])
    y_arr = np.concatenate([y_arr, np.full(3*len(result[i]), i)])

## 各イントロンの3'末尾の配列と、特定の配列のマッチング率の算出

In [ ]:
# # 3'側末端に近いほど高いスコアを与える
# # 1塩基離れるごとにスコアを1/2にする
# def match(seq1, seq2):
#     scores = [(0.9)**i for i in range(len(seq1))]
#     score = 0
#     for i,j,k in zip(seq1, seq2, scores):
#         if i == j:
#             score += k
#     return round(score,2)

# # 3'側末端に近いほど高いスコアを与える
# # 3'末端に最も高いスコアを与え、1塩基離れるごとにスコアを1減らす
# def match(seq1, seq2):
#     scores = [len(seq1)-i for i in range(len(seq1))]
#     score = 0
#     for i,j,k in zip(seq1, seq2, scores):
#         if i == j:
#             score += k
#     return round(score,2)

# 各塩基のマッチングに対して均等にスコアリングする
def match(seq1, seq2):
    match = 0
    for i,j in zip(seq1, seq2):
        if i == j:
            match += 1
    return round(match/len(seq1),2)

mtf = "AAAAATACTAATATCTTTCAAG"
# mtf = "ACTAATATCTTTCAAG"
# mtf = "CAAAG"
seq = gbk.gDNA_seq()
match_result = []
for i in gbk.intron_list():
    tmp_seq = seq[i[0]:i[1]][-len(mtf):]
    match_result.append(match(mtf, tmp_seq))


## 各イントロンにおける5'側および3'側３塩基パターンが"GTA"、"AGG"であるイントロンの分布

In [ ]:

# 各イントロンにおける5'側３塩基パターンが"GTA"のイントロンの分布
motif_gta = "GTA"
introns5 = []
for i in range(gbk.intron_num()):
    if gbk.intron_seq(i)[:len(motif_gta)] == motif_gta:
        introns5.append(1)
    else:
        introns5.append(0)
df_gta = pd.DataFrame({"intron":[i+1 for i in range(len(introns5))], "existence":introns5, "description": [f"intron: {i+1}" for i in range(len(introns5))]})

# 各イントロンにおける3'側３塩基パターンが"AAG"のイントロンの分布
motif_aag= "AAG"
introns3 = []
for i in range(gbk.intron_num()):
    if gbk.intron_seq(i)[-len(motif_aag):] == motif_aag:
        introns3.append(1)
    else:
        introns3.append(0)
df_aag = pd.DataFrame({"intron":[i+1 for i in range(len(introns3))], "existence":introns3, "description": [f"intron: {i+1}" for i in range(len(introns3))]})


## TTN遺伝子全長における脊椎動物種間のエクソンおよびイントロンの保存性平均値の算出

In [ ]:

with open('../data/gbk/human_ttn.gb') as f:
    lines = f.readlines()
    for i in lines[:10]:
        for j in i.split():
            if "complement" in j:
                start, end = j.replace("complement(", "").replace(")", "").split("..")
start = int(start)
end = int(end)

## UCSCのTbale BrowserからダウンロードしたphastCons100wayのデータの読み込み
# 3分割したファイルの読み込みと統合
ttn1 = pd.read_csv('../data/ucsc/ucsc_ttn1.txt', sep='\t', skiprows=9, nrows=100000, header=None)
ttn2 = pd.read_csv('../data/ucsc/ucsc_ttn2.txt', sep='\t', skiprows=9, nrows=100000, header=None)
ttn3 = pd.read_csv('../data/ucsc/ucsc_ttn3.txt', sep='\t', skiprows=9, nrows=100000, header=None)
ttn = pd.concat([ttn1, ttn2, ttn3])

# TTN遺伝子範囲のみ抽出
ucsc_ttn = ttn[(ttn[0] >= start) & (ttn[0] <= end)]
ucsc_ttn.columns = ['pos','cons' ]

# 読み込んだデータに一部欠損があるために、欠損箇所を埋める
tmp = pd.DataFrame({"pos": np.arange(178525989, 178807423)})
df = pd.merge(tmp, ucsc_ttn, left_on='pos', right_on='pos', how='left')

## 各イントロンとエクソンの塩基ごとの保存性の平均値を求める

#　TTNのmRNAは逆鎖にあるため、反転させる
df = df.sort_values('pos', ascending=False)
df['pos'] = df['pos'].apply(lambda x: 178807422-x)
df = df.set_index('pos', drop=True)

# エクソンのごとの保存性の平均値を求める
exon_cons_ave = {}
for i in range(gbk.exon_num()):
    r = gbk.exon_list()[i]
    ave = round(df.iloc[r[0]:r[1], 0].mean(),2)
    exon_cons_ave[i+1] = ave

# イントロンのごとの保存性の平均値を求める
intron_cons_ave = {}
for i in range(gbk.intron_num()):
    r = gbk.intron_list()[i]
    ave = round(df.iloc[r[0]:r[1], 0].mean(),2)
    intron_cons_ave[i+1] = ave

## TTN遺伝子全長における脊椎動物種間のイントロン両末端n塩基の保存性平均値の算出

In [ ]:
# 検討するイントロン末端からの塩基数
n = 80

# 5'側のイントロンn塩基の保存性の平均
left_intron_cons = {} 
for num, i in enumerate(gbk.intron_list()):
    left_intron_cons[num+1] = list(df.iloc[i[0]:i[0]+n,0])
left_intron_cons_ave_df = pd.DataFrame.from_dict(left_intron_cons)
left_intron_cons_ave_df.index = [i for i in range(1, n+1)]
left_intron_cons_ave_df = left_intron_cons_ave_df.apply(lambda x: round(x,2))

# 5'側のイントロンn塩基の保存性の平均
right_intron_cons = {} 
for num, i in enumerate(gbk.intron_list()):
    right_intron_cons[num+1] = list(df.iloc[i[1]-n:i[1],0])
right_intron_cons_ave_df = pd.DataFrame.from_dict(right_intron_cons)
right_intron_cons_ave_df.index = [-i for i in range(n, 0, -1)]
right_intron_cons_ave_df = right_intron_cons_ave_df.apply(lambda x: round(x,2))

## 上記情報の可視化

In [ ]:
# グラフ作成
def fig_show_all():
    threshold = 0.5
    titles = ["3の倍数の塩基数のエクソン(A)", "スプライスされたイントロンの起点の数(B)", "スプライスされたイントロンの長さ(C)", "スプライスされたイントロンの終点の数(D)",f"3'側イントロン末端配列と{mtf}配列とのマッチングスコア(E)",
            "5'側末端の塩基配列がGTAのイントロン(F)","3'側末端の塩基配列がAAGのイントロン(G)", "各エクソンの塩基の保存性の平均値(H)", "各イントロンの塩基の保存性の平均値(I)", f"各イントロンの5'側の{n}塩基の保存性(J)", f"各イントロンの3'側の{n}塩基の保存性(K)"]
    fig = make_subplots(rows=11, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=titles)
    fig.add_trace(go.Bar(x=df_3x['exon'], y=df_3x['existence'], customdata=df_3x['length'], hovertemplate="Intron%{x}<br>Lenght: %{customdata}<extra></extra>"), row=1, col=1)
    fig.add_trace(go.Bar(x=sdf.index, y=sdf[0], hovertemplate='Intron%{x}<extra></extra>'), row=2, col=1)
    fig.add_trace(go.Scatter(x=x_arr, y=y_arr, hovertemplate='Intron edge: Intron%{x}<extra></extra>'), row=3, col=1)
    fig.add_trace(go.Bar(x=edf.index, y=edf[0], hovertemplate='Intron%{x}<extra></extra>'), row=4, col=1)
    fig.add_trace(go.Bar(x=[i+1 for i in range(len(match_result))], y=match_result, hovertemplate='Intron%{x}<br>Match: %{y}<extra></extra>'), row=5, col=1)
    fig.add_trace(go.Scatter(x=[1,gbk.intron_num()], y=[threshold, threshold],line=dict(color='red', dash='dot')), row=5, col=1)
    fig.add_trace(go.Bar(x=df_gta['intron'], y=df_gta['existence'], hovertemplate='Intron: %{x}<extra></extra>'), row=6, col=1)
    fig.add_trace(go.Bar(x=df_aag['intron'], y=df_aag['existence'], hovertemplate='Intron: %{x}<extra></extra>'), row=7, col=1)
    fig.add_trace(go.Bar(x=list(exon_cons_ave.keys()), y=list(exon_cons_ave.values()), hovertemplate= 'Exon: %{x}<br>Conservation: %{y}<extra></extra>'), row=8, col=1)
    fig.add_trace(go.Bar(x=list(intron_cons_ave.keys()), y=list(intron_cons_ave.values()), hovertemplate= 'Intron: %{x}<br>Conservation: %{y}<extra></extra>'), row=9, col=1)
    fig.add_trace(go.Heatmap(x=left_intron_cons_ave_df.columns, y=left_intron_cons_ave_df.index, z=left_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate='Intron: %{x}<br>Base: %{y}<br>Conservation: %{z}<extra></extra>', showscale=False), row=10, col=1)
    fig.add_trace(go.Heatmap(x=right_intron_cons_ave_df.columns, y=right_intron_cons_ave_df.index, z=right_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate='Intron: %{x}<br>Base: %{y}<br>Conservation: %{z}<extra></extra>', showscale=False), row=11, col=1)
    fig.update_yaxes(range=[0, 1], row=8, col=1)
    fig.update_yaxes(range=[0, 1], row=9, col=1)
    fig.update_yaxes(autorange="reversed", row=11, col=1)

    # 各サブプロットのタイトルを追加
    fig.update_layout(width=1500, height=1500, title_text=f"各種情報の比較 (NM_001267550.2)", showlegend=False)
    # サブプロットタイトルの位置、フォントサイズの変更
    for i in range(len(fig.layout.annotations)):
        fig.layout.annotations[i].font.size = 12
        fig.layout.annotations[i].x = 0
        fig.layout.annotations[i].xanchor = 'left'
    fig.show()

## 任意のグラフのみを表示
**例** 
show_fig(list("ABC")): A、B、Cのグラフを順に表示

In [ ]:
def show_fig(lst):
    # 設定値
    threshold = 0.5
    title_dic = {"A":"3の倍数の塩基数のエクソン", "B":"スプライスされたイントロンの起点の数", "C":"スプライスされたイントロンの長さ", 
                  "D":"スプライスされたイントロンの終点の数","E":f"3'側イントロン末端配列と{mtf}配列とのマッチングスコア(E)",
                  "F":"5'側末端の塩基配列がGTAのイントロン","G":"3'側末端の塩基配列がAAGのイントロン", "H":"各エクソンの塩基の保存性の平均値(H)", 
                  "I":"各イントロンの塩基の保存性の平均値(I)", "J":f"各イントロンの5'側の{n}塩基の保存性(J)", "K":f"各イントロンの3'側の{n}塩基の保存性"}
    title_list = [title_dic[i] for i in lst]
    fig = make_subplots(rows=len(lst), cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=title_list)
    r = 1
    for i in lst:
        if i == "A":
            exec("fig.add_trace(go.Bar(x=df_3x['exon'], y=df_3x['existence'], customdata=df_3x['length'], hovertemplate='Intron%{x}<br>Lenght: %{customdata}<extra></extra>'), row=r, col=1)")
            r += 1
        elif i == "B":
            exec("fig.add_trace(go.Bar(x=sdf.index, y=sdf[0], hovertemplate='Intron%{x}<extra></extra>'), row=r, col=1)")
            r += 1
        elif i == "C":
            exec("fig.add_trace(go.Scatter(x=x_arr, y=y_arr, hovertemplate='Intron edge: Intron%{x}<extra></extra>'), row=r, col=1)")
            r += 1  
        elif i == "D":
            exec("fig.add_trace(go.Bar(x=edf.index, y=edf[0], hovertemplate='Intron%{x}<extra></extra>'), row=r, col=1)")
            r += 1
        elif i == "E":
            exec("fig.add_trace(go.Bar(x=[i+1 for i in range(len(match_result))], y=match_result, hovertemplate='Intron%{x}<br>Match: %{y}<extra></extra>'), row=r, col=1)")
            exec("fig.add_trace(go.Scatter(x=[1,gbk.intron_num()], y=[threshold, threshold],line=dict(color='red', dash='dot')), row=r, col=1)")
            r += 1  
        elif i == "F":
            exec("fig.add_trace(go.Bar(x=df_gta['intron'], y=df_gta['existence'], hovertemplate='Intron: %{x}<extra></extra>'), row=r, col=1)")
            r += 1
        elif i == "G":
            exec("fig.add_trace(go.Bar(x=df_aag['intron'], y=df_aag['existence'], hovertemplate='Intron: %{x}<extra></extra>'), row=r, col=1)")
            r += 1
        elif i == "H":
            exec("fig.add_trace(go.Bar(x=list(exon_cons_ave.keys()), y=list(exon_cons_ave.values()), hovertemplate= 'Exon: %{x}<br>Conservation: %{y}<extra></extra>'), row=r, col=1)")
            exec("fig.update_yaxes(range=[0, 1], row=r, col=1)")
            r += 1
        elif i == "I":
            exec("fig.add_trace(go.Bar(x=list(intron_cons_ave.keys()), y=list(intron_cons_ave.values()), hovertemplate= 'Intron: %{x}<br>Conservation: %{y}<extra></extra>'), row=r, col=1)")
            exec("fig.update_yaxes(range=[0, 1], row=r, col=1)")
            r += 1
        elif i == "J":
            exec("fig.add_trace(go.Heatmap(x=left_intron_cons_ave_df.columns, y=left_intron_cons_ave_df.index, z=left_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate='Intron: %{x}<br>Base: %{y}<br>Conservation: %{z}<extra></extra>', showscale=False), row=r, col=1)")
            r += 1
        elif i == "K":
            exec("fig.add_trace(go.Heatmap(x=right_intron_cons_ave_df.columns, y=right_intron_cons_ave_df.index, z=right_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate='Intron: %{x}<br>Base: %{y}<br>Conservation: %{z}<extra></extra>', showscale=False), row=r, col=1)")
            exec("fig.update_yaxes(autorange='reversed', row=r, col=1)")
            r += 1
            
    fig.update_layout(width=1500, height=150*len(lst), title_text=f"各種情報の比較 (NM_001267550.2)", showlegend=False)
    # サブプロットタイトルの位置、フォントサイズの変更
    for i in range(len(fig.layout.annotations)):
        fig.layout.annotations[i].font.size = 12
        fig.layout.annotations[i].x = 0
        fig.layout.annotations[i].xanchor = 'left'
    fig.show()

In [ ]:
fig_show_all()

In [ ]:
show_fig(list("AJCK"))